In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup
import requests
import csv
from selenium.webdriver.common.action_chains import ActionChains
import urllib
import xlwt
wb = xlwt.Workbook()

country_list = ["DETS 2019", "DETS 2018", "DESS 2017", "DESS 2016", "DESS 2015", "DESS 2014", "DESS 2013", "DESS 2012", "DESS 2011", "DESS 2010", "DESS 2009", "DESS 2008"]
# country_list = ["DESS 2010"]

chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome("T:\\Universum Research\\2019\\Training\\Confirmit Export\\chromedriver",chrome_options=chromeOptions, desired_capabilities=chromeOptions.to_capabilities())
driver.maximize_window()
    
driver.get("https://tabulator.universumglobal.com/ui/default.aspx") 
time.sleep(2)

obj = driver.switch_to.alert
time.sleep(2)
obj.dismiss()

username = driver.find_element_by_id("txtUserName")
password = driver.find_element_by_id("txtPassword")
username.send_keys("pantheon\surendra.kumar")
password.send_keys("Feb@2019")
login_attempt = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[2]/td[2]/table/tbody/tr[8]/td/table/tbody/tr/td/div/a")
login_attempt.submit()

try:
    for country_name in country_list:

    ###survey searching

        country = country_name
        x_path = "//td[contains(text(),'" + country + "')]"
        search_survey = driver.find_element_by_xpath(x_path)
        search_survey.click()
        time.sleep(1)

        ########################### FRAMES ##############################
        iframeQuestionGroups = driver.find_element_by_id('iframeQuestionGroups')
        iframeQuestions = driver.find_element_by_id('iframeQuestions')
        iframeAnalyze = driver.find_element_by_id('iframeAnalyze')
        iframeCategorize = driver.find_element_by_id('iframeCategorize')
        iframeAlternatives = driver.find_element_by_id('iframeAlternatives')
        iframeFilter = driver.find_element_by_id('iframeFilter')
        
        ############################# Questions Group frame###########################
        driver.switch_to.frame(iframeQuestionGroups)        

        if ("Official Target Groups" in driver.page_source):
            driver.find_element_by_xpath("//span[text() = 'Official Target Groups']").click() ##click Official Target Groups            
        elif ("Educational Profile" in driver.page_source):
            driver.find_element_by_xpath("//span[text() = 'Educational Profile']").click()  ##click Educational Profile            

        time.sleep(1)
        driver.switch_to.default_content()

        ############################# Alternatives frame double click to add FILTER ###########################
        actionChains = ActionChains(driver)

        driver.switch_to.frame(iframeAlternatives)
        
        time.sleep(1)
        
        flag_1 = ""
        
        if ("Student" in driver.page_source): 
            add_filter= driver.find_element_by_xpath("//option[contains(text(), 'Student')]")  ##add student to filter 
            actionChains.double_click(add_filter).perform()
        elif ("Undergraduate" in driver.page_source): 
            add_filter= driver.find_element_by_xpath("//option[contains(text(), 'Undergraduate')]")  ##add Undergraduate to filter
            actionChains.double_click(add_filter).perform()         
        else:
            flag_1 = 'not found'
        driver.switch_to.default_content()
        
        ############################# Questions Group frame###########################
#         driver.switch_to.frame(iframeQuestionGroups)


#         if ("Official Target Groups" in driver.page_source):
#             driver.find_element_by_xpath("//span[text() = 'Official Target Groups']").click() ##click Official Target Groups            
#         elif ("Educational Profile" in driver.page_source):
#             driver.find_element_by_xpath("//span[text() = 'Educational Profile']").click()  ##click Educational Profile            

#         time.sleep(3)
#         driver.switch_to.default_content()
        
        ############################# Questions frame ###########################
        
        driver.switch_to.frame(iframeQuestions)
        time.sleep(3)
        select_element = Select(driver.find_element_by_id("listQuestions"))
        select_element.deselect_by_index(0)  # select and unselect Main field of study & official target groups
        #select_element.select_by_index(1)
        if ("2. Main Field of Study" in driver.page_source):
            select_element.select_by_visible_text("2. Main Field of Study")
        elif ("Main Field of Study" in driver.page_source):
             select_element.select_by_visible_text("Main Field of Study")
        elif ("Field of study" in driver.page_source):      
             select_element.select_by_visible_text("Field of study") 
        #select_element.select_by_value("Q_0_563317_7_What is your main field of study? / What was your main field of study?")
        #select_element.deselect_by_value("Q_0_563316_3_Official Target Groups")        
        driver.switch_to.default_content()                
        
        ############################# Add to Categorize ###########################
        driver.switch_to.frame(iframeCategorize)
        
        time.sleep(3)
        driver.find_element_by_xpath("//a[@onclick ='parent.AddToList(document.forms[0].listCategorize);']").click()
        
        driver.switch_to.default_content()
        
        ############################# Questions Group frame ###########################
        driver.switch_to.frame(iframeQuestionGroups)
        
        driver.find_element_by_xpath("//span[text() = 'Rankings']").click()        
        time.sleep(3)
        driver.switch_to.default_content()

        ############################# Add to Analyze ###########################
        driver.switch_to.frame(iframeAnalyze)        
        time.sleep(1)
        driver.find_element_by_xpath("//a[@onclick ='parent.AddToList(document.forms[0].listAnalyze);']").click()
        
        time.sleep(1)
        driver.switch_to.default_content()

        ############################# Get Table ###########################
        parent_window_handle = driver.current_window_handle
        first_window = driver.window_handles[0]

        driver.find_element_by_xpath("//a[@onclick ='getTable(false)']").click()        

        time.sleep(10)

        ############################ POPUP WINDOWs For data ###########################
        handles = driver.window_handles;

        driver.switch_to.window(handles[1])
        print(driver.title)
        print(driver.find_element_by_xpath("//a[@onclick ='saveSettings()']").text)
        print(driver.find_element_by_xpath("//a[@onclick ='execute(false)']").text)

        ############################ aply setting HTML format in popup window ###########################

        time.sleep(1)

        #### Output tab
        driver.find_element_by_xpath("//em[text() = 'Output']").click()
        time.sleep(1)
        driver.find_element_by_xpath("//select[@name='listOutput']/option[text()='HTML Table']").click()
        time.sleep(1)
        ### General tables tab
        driver.find_element_by_xpath("//em[text() = 'General Tables']").click()
        time.sleep(1)
        driver.find_element_by_xpath("//*[@id='SpanColumnPercent']/label").click()
        driver.find_element_by_xpath("//*[@id='SpanColRank']/label").click()
        time.sleep(1)        
        ######## weighting
        driver.find_element_by_xpath("//em[text() = 'Weighting']").click()  ###
        driver.find_element_by_xpath("//input[@id = 'chkAutomaticWeight']").click()
        driver.find_element_by_xpath("//select[@name='listWeight']/option[text()='Weighted All']").click()   ### Weighted Total ## Weighted All ##No Weight                    
        time.sleep(1)
        ### Apply settings tab
        driver.find_element_by_xpath("//a[text() ='Apply Settings']").click()
        time.sleep(1)

        ###iframe for html data
        iframes_data = driver.find_element_by_xpath("//iframe[contains(@src, 'showData.aspx?guid')]")
        driver.switch_to.frame(iframes_data)

        res = driver.execute_script("return document.documentElement.outerHTML")
        soup = BeautifulSoup(res, 'lxml')

        ################ Put into EXcel##################
        ws = wb.add_sheet(country)
        table = soup.findAll('tbody')[1]  
        rows = table.findAll("tr")
        x = 0
        for tr in rows:
            cols = tr.findAll("td")
            if not cols: 
                # when we hit an empty row, we should not print anything to the workbook
                continue
            y = 0
            for td in cols:
                texte_bu = td.text
                texte_bu = texte_bu.encode('utf-8')
                texte_bu = texte_bu.strip()
                if x <= 4:
                    ws.write(x, y, td.text)
                elif x == 5:    
                    ws.write(x, y + 2, td.text)
                elif x == 6:    
                    ws.write(x, y, td.text)    
                else:
                    ws.write(x, y + 1, td.text) 
                y = y + 1
            # update the row pointer AFTER a row has been printed
            # this avoids the blank row at the top of your table
            x = x + 1

        wb.save('rankings_DE.xls')
        driver.switch_to.window(handles[0])
        driver.find_element_by_xpath("//a[@href ='chooseSurvey.aspx']").click()
        time.sleep(3)
except Exception as ex:
    print(ex)
    driver.execute_script("alert('An Error Occured: please review output information in console');")
# driver.quit()     

C:\Users\kumarsur\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


Iframes....
Analyze Data
Save Settings
Apply Settings
Iframes....
Analyze Data
Save Settings
Apply Settings
Iframes....
Analyze Data
Save Settings
Apply Settings
Iframes....
Analyze Data
Save Settings
Apply Settings
Iframes....
Analyze Data
Save Settings
Apply Settings
Iframes....
Analyze Data
Save Settings
Apply Settings
Iframes....
Analyze Data
Save Settings
Apply Settings
Iframes....
Analyze Data
Save Settings
Apply Settings
Iframes....
Analyze Data
Save Settings
Apply Settings
Iframes....
Analyze Data
Save Settings
Apply Settings
Iframes....
Analyze Data
Save Settings
Apply Settings
Iframes....
Analyze Data
Save Settings
Apply Settings
